In [1]:
'''首先准备数据到tensorflow可以读取的格式'''
'''
1.分词
2.词语转化成ID去表示：embedding，词语长度转化为向量
    定义一个矩阵，第一个维度是词语长度|V|，诶二个维度是embedding_size
3.需要索引：
        词语A-->id(5)词语对应的embedding是哪一行的数据
4.为了达到上述目标需要将词表统计出来   

5.label统计出来，也变成ID-->
'''

'首先准备数据到tensorflow可以读取的格式'

In [1]:
import sys
import os
import jieba

In [2]:
#输入文件
train_file = './cnews_data/cnews.train.txt'
val_file = './cnews_data/cnews.val.txt'
test_file = './cnews_data/cnews.test.txt'

#输出文件  
#分词后文件
seg_train_file = './cnews_data/cnews.train.seg.txt'
seg_val_file = './cnews_data/cnews.val.seg.txt'
seg_test_file = './cnews_data/cnews.test.seg.txt'

#此表文件：词语到ID的转化的映射
vocab_file = './cnews_data/cnews.covab.txt'
#label到ID的映射
category_file = './cnews_data/cnews.category.txt'



In [3]:
def generate_seg_file(input_file,output_seg_file):
    '''分词函数：按行对input——file的内容部分进行分词'''
    with open(input_file,'r') as f:
        lines = f.readlines()
    with open(output_seg_file,'w') as f :
        for line in lines:
            label , content = line.strip('\r\n').split('\t')
            word_iter = jieba.cut(content) 
            word_content = ' '
            for word in word_iter:#出现空格被分成一个字符
                word = word.strip()
                if word != '':
                    word_content += word + '   '
            out_line = '%s\t%s\n'%(label, word_content.strip('  '))#将最后的多余空格删除
            f.write(out_line)    

generate_seg_file(train_file, seg_train_file)
generate_seg_file(val_file, seg_val_file)
generate_seg_file(test_file, seg_test_file)


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/4d/vrf7sljd17q74bxt50lvhxsw0000gn/T/jieba.cache
Loading model cost 0.629 seconds.
Prefix dict has been built succesfully.


In [4]:
'''词表构建，把分词后的结果文件统计出来生成词表'''
def generate_vocab_file(input_seg_file, output_vocab_file):
    with open(input_seg_file,'r') as f:
        lines = f.readlines()
    word_dict={} #用字典来保存词语的频次信息
    for line in lines:
        label, content = line.strip('\r\n').split('\t')
        for word in content.split():
            word_dict.setdefault(word, 0)#设计默认词频，如果这个词语不在字典中就默认为0，如果有的话这个setdefault函数就不会起作用
            word_dict[word] += 1
    #[(word,frequence),(   ,    )]
    sorted_word_dict = sorted(word_dict.items(), key=lambda x : x[1], reverse=True )
    with open(output_vocab_file, 'w') as f:
        f.write('<UNW>\t10000\n')#在测试集中可能有些词在训练集中并没有导致找不到这个时候就返回UNK的id，在词频过滤中有些词被过滤掉也找不到。为了应付这种情况
        for item in sorted_word_dict:
            f.write('%s\t%d\n' % (item[0], item[1]))
    
    

generate_vocab_file(seg_train_file,vocab_file)            

In [5]:
'''把出现一次的词给删除掉。毕竟神经网络模型是一个概率模型。只出现一次没什么帮助'''
def generate_category_dict(input_file, category_file):
    with open(input_file,'r') as f:
        lines = f.readlines()
    category_dict = {}
    word_dict={} #用字典来保存词语的频次信息
    for line in lines:
        label, content = line.strip('\r\n').split('\t')    
        category_dict.setdefault(label, 0)
        category_dict[label] += 1
    category_number = len(category_dict)
    with open(category_file,'w') as f:
        for category in category_dict:
            line = '%s\n' % category
            print('%s\t%d' % (category, category_dict[category]))
            f.write(line)
generate_category_dict(train_file, category_file)
'''共10类，每类5000个样本，共5万个样本'''

体育	5000
娱乐	5000
家居	5000
房产	5000
教育	5000
时尚	5000
时政	5000
游戏	5000
科技	5000
财经	5000


'共10类，每类5000个样本，共5万个样本'

In [6]:
'''预处理结束，正式进入 实现LSTM实现文本分类模型的环节'''

'预处理结束，正式进入 实现LSTM实现文本分类模型的环节'